In [ ]:
%matplotlib inline
import cv2
from pylab import *
import torch
from torch import nn

# https://pypi.org/project/models/0.9.3/
from models import LinkNet34

import torchvision.transforms as transforms
from torch.autograd import Variable
import sys
from PIL import Image, ImageFilter


In [ ]:
# linking gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# initializing linknet34 
# https://nikolasent.github.io/deeplearning/competitions/2018/06/05/Multiclass-semantic-segmentation-with-LinkNet34.html
model = LinkNet34()
# model.load_state_dict(torch.load('linknet.pth'))
model.load_state_dict(torch.load('linknet.pth', map_location=lambda storage, loc: storage))

model.eval()
model.to(device)
# summary(model, (3, 256, 256))
1

# to handle image manipulations.. 
# if you want to do multiple image manipulations, you could add all the image manipulations inside 
# transforms.Compose
img_transform = transforms.Compose([
    transforms.Resize((256,256)),
     # transform the image into tensor, normalize the pixel values in range 0-1
    transforms.ToTensor(),
    # setting values for normalization, normalize the image using standard deviation
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False) 
])
t = transforms.Resize(size=(256,256))


def plt_mask(path):
    _img = Image.open(path)
    a = img_transform(_img)
    a = a.unsqueeze(0)
    imgs = Variable(a.to(device))
    pred = model(imgs)
    img = np.array(_img)
    mask = pred > 0.5
    mask = mask.squeeze()
    mask = mask.cpu().numpy()
    img=np.array(t(Image.fromarray(img)))
    img[mask==0]=170
#     print(shape(img))
    return img;
#     plt.imshow(img)

images = os.listdir('/home/rt/Downloads/FacialExpressionDataset/FacialExpressions_800_800/')
dirName = '/home/rt/Downloads/FacialExpressionDataset/FacialExpressions_800_800/'
model.eval()

faces = []
# faces.append(plt_mask(dirName + images[0]))
for img in images:
    faces.append(plt_mask(dirName + img))
plt.imshow(faces[0])

